# Simple Qubit Flipping Example
This example explains how we can flip one or more qubits using qiskit

In [ ]:
from qiskit_iqm import IQMProvider
from qiskit import QuantumCircuit, QuantumRegister, execute

To access Helmi, we connect to the backend using the cocos url

In [ ]:
provider = IQMProvider("https://qc.vtt.fi/cocos")
backend = provider.get_backend()

## Creating circuits

First, we define a function that creates a circuit with one qubit and applies X-gate to the qubit. The X-gate flips the qubit from |0> to |1>. After applying the gate we measure and return the ciruit with the correct mapping

In [ ]:
def single_flip_circuit(qubit: int) -> tuple[QuantumCircuit, dict]:
    qreg = QuantumRegister(1, "qb")
    circuit = QuantumCircuit(qreg)
    circuit.x(0)
    circuit.measure_all()
    qubit_mapping = {qreg[0]: qubit}
    return circuit, qubit_mapping

Next we define a function that takes a list as a parameter. Then creates a circuit with number of qubits corresponding to the length of the list. After creating the circuit, we apply an X-gate to every qubit in the circuit. We measure the circuit and map the qubits. 

In [ ]:
def flip_circuit(qubits: list[int]) -> tuple[QuantumCircuit, dict]:
    qreg = QuantumRegister(len(qubits), "qb")
    circuit = QuantumCircuit(qreg)
    for qubit in qubits:
        circuit.x(qubit)
    circuit.measure_all()
    qubits_mapping = {qreg[i]: qubits[i] for i in range(len(qubits))}
    return circuit, qubits_mapping

Calculating the probability of the desired states from the job result:

In [ ]:
def calculate_success_probability(counts: dict, shots: int, desired_state: str) -> float:
    success_counts = counts.get(desired_state, 0)
    return success_counts / shots

## Submitting the Jobs

In [ ]:
def flip_single_qubit(desired_state: str, shots: int = 1000) -> None:
    print("\nFlip one qubit at a time\n")
    for qb in range(5):
        circuit, mapping = single_flip_circuit(qb)
        job = execute(circuit, backend, shots=shots, initial_layout=mapping)
        counts = job.result().get_counts()
        success_probability = calculate_success_probability(counts, shots, desired_state)
        print(
            f"QB{qb + 1} -> {counts}, Success probability: {success_probability * 100:.2f}%",
        )

In [ ]:
flip_single_qubit('1')

In [ ]:
def flip_qubits(desired_state: str, shots: int = 1000) -> None:
    print("\nFlip all qubits at once\n")
    circuit, mapping = flip_circuit([0, 1, 2, 3, 4])
    job = execute(circuit, backend, shots=shots, initial_layout=mapping)
    counts = job.result().get_counts()
    success_probability = calculate_success_probability(counts, shots, desired_state)
    print(
        f"Counts: {counts}, \nSuccess probability: {success_probability * 100:.2f}%",
    )

In [ ]:
flip_qubits('11111', 1000)